# **SDSE Final Project: Predicting Car Fuel Efficiency with Machine Learning**
Group #(insert number here)

Our notebook attempts to predict a given car's **combined fuel efficiency (MPG)** based on information that is available **before** buying the car:

- Make and type                 (SUV, sedan, etc.)
- Drivetrain                    (FWD, AWD, etc.)
- Engine displacement           (L)
- Number of cylinders           (dimensinoless)
- Transmission type             (automatic, manual, etc.)

Why this matters:

- Consumers can estimate real-world fuel costs for a car model even when independent test results are not widely available yet, such with new vehicle models and trims.
- Dealerships and fleet managers can compare many options quickly based on expected efficiency.
- Policy or sustainability teams can more immediately simulate how changing the mix of vehicles (more small engines, more hybrids, etc.) might affect fuel consumption and greenhouse/noise emissions.

The notebook follows the outline specified for the project submissions in the lab session. 

0. Choose performance metric
1. load data
2. split off the test data set
3. Choose the families of model and hyperparameter variations to test.
4. Execute (train and evaluate) models
5. Choose best model and evaluate with test data.

# 0. Performance metric:

Since we are working with a regression model, we will be using the R^2 coefficient of determination as the metric to evaluate all our models.

## 1. Load Data
In this section we:


- **1.1**: import necessary libraries/packages
    - **pandas / numpy** for data manipulation  
    - **matplotlib** for plotting  
    - **scikit‑learn** tools for preprocessing and linear regression  
    - **TensorFlow / Keras** for building neural networks
- read in the data
- clean up the data
- plot data
- reduce number of classes in categories where there are too many (consolidation)





In [ ]:
# Imports
# !pip install tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization


In [ ]:
# Load in raw dataset with specs + fuel efficiency
data = pd.read_csv("car_data.csv")
# print("First five rows of data: \n", data.head(5))     # Uncomment if you want a quick peak 

**1.3: Defining Feature Columns and Target**

Next, we decide which columns will be used as inputs (features) and which column is the output (target).

- `categorical` lists all string‑based variables we might want to explore.
- `numerical` contains the numeric engine characteristics that are important for fuel use.
- `output` is the target: combined MPG.
- `categorical_for_model` is a refined list of categorical columns used in the actual model.  
  We keep a fixed set of five categorical variables as required by the project guidelines, and avoid overly specific identifiers which would be difficult to generalize well, such as the model name.

In [ ]:
# Define which columns are inputs (X) and which is the target (y)

# Raw categorical columns present in the dataset
categorical = ['type', 'drive', 'make', 'model', 'transmission']

# Raw numerical columns
numerical = ['cylinders', 'displacement']

# Target we want to predict
output = 'combination_mpg'

# Categorical columns we’ll actually use in the model
# (we drop 'model' here to avoid too many one-hot columns)
categorical_for_model = ['type', 'drive', 'fuel_type', 'make', 'transmission']

# Final feature matrix and target vector
X = data[categorical_for_model + numerical]
y = data[output]


**1.4: Data Cleaning and Preparation**

Before training any model, our code handles missing values:

1. **Drop rows with missing target**:  
   If `combination_mpg` is missing, the data is unusable for supervised learning.

2. **Impute numerical features with the mean**:  
   This maintains a baseline that keeps all existing data while avoiding bias towards any existing value.

3. **Impute categorical features with the mode** (most frequent category):  
   This preserves the most likely class and keeps categories consistent.

After cleaning, our code builds:

- `feature_cols`: all columns that will be used as inputs
- `X`: the feature matrix  
- `y`: the target vector

In [ ]:
# Data preparation
data = data.dropna(subset=[output])

for col in numerical:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].mean())

for col in categorical_for_model:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].mode()[0])

# # Defining feature matrix X and target y
# feature_cols = categorical_for_model + numerical
# X = data[feature_cols]
# y = data[output]

# print("\nFeature columns used for the model:")
# print(feature_cols)
# print(f"\nNumber of samples after cleaning: {len(X)}")

**1.5: Consolidating categories with too many classes**

We will consolidate the categories "make" and "type". We will ignore the "model" data in the model since each entry is a different model.

The "make" category will be reduced to 3 classes by region of the company:
1. **Asia:** kia, hyundai, genesis, mazda, honda, acura, subaru, mitsubishi, toyota, nissan, infiniti
2. **Europe:** bmw, jaguar, mini, audi, land rover, volvo, volkswagen, aston martin, porsche, bentley, mercedes-benz
3. **America:** chevrolet, jeep, gmc, ford, cadillac, buick, ram, roush performance, chrysler

The "type" category will be reduced to 3 classes by size:
1. **Small:** small sport utility vehicle, subcompact car, compact car, two seater, minicompact car, small station wagon
2. **Medium:** midsize car, standard sport utility vehicle, small pickup truck, midsize station wagon
3. **Large:** large car, minivan, standard pickup truck


In [ ]:
# defining region lists for make region categories
asia = ["kia", "hyundai", "genesis", "mazda", "honda", "acura",
    "subaru", "mitsubishi", "toyota", "nissan", "infiniti"]

europe = ["bmw", "jaguar", "mini", "audi", "land rover", "volvo",
    "volkswagen", "aston martin", "porsche", "bentley", "mercedes-benz"]

america = ["chevrolet", "jeep", "gmc", "ford", "cadillac", "buick",
    "ram", "roush performance", "chrysler"]

def consolidate_region(make):
    if make in asia:
        return "asia"
    if make in europe:
        return "europe"
    if make in america:
        return "america"
    return "other"   # just in case

# consolidating make data to make_region
data["make_region"] = data["make"].apply(consolidate_region)

small = ["small sport utility vehicle","subcompact car","compact car",
            "two seater","minicompact car","small station wagon"]
medium = ["midsize car", "standard sport utility vehicle", 
            "small pickup truck", "midsize station wagon"]
large = ["large car", "minivan", "standard pickup truck"]

def consolidate_size(type):
    if type in small:
        return "small"
    if type in medium:
        return "medium"
    if type in large:
        return "large"
    return "other"
        
# consolidating type data to size
data["size"] = data["type"].apply(consolidate_size)


**1.6: Defining the final feature set for modelling**

In [ ]:
# ----- Selecting the final feature set for modeling -----

# Consolidated list of categorical features we'll use.
# Cleaned versions of raw columns so the model
# doesn’t explode into too many one-hot encoded columns.
categorical_for_model_consolidated = [
    'size',
    'drive',
    'fuel_type',
    'make_region',
    'transmission'
]

# Combine categorical + numerical features to form the full input (X)
feature_cols = categorical_for_model_consolidated + numerical

# X contains only the columns we want the model to learn from
X = data[feature_cols]

# y is the target we want to predict — combined MPG
y = data[output]

# Helpful summary printout
print("\nFeature columns used for the model:")
print(feature_cols)

print(f"\nNumber of samples after cleaning: {len(X)}")


## 2. Split off the test dataset:

In [ ]:
# Split into train and test sets
# - Train: used to fit / tune models
# - Test: held out until the very end to estimate real-world performance
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,     # 20% of rows are used for final evaluation
    random_state=42    # fixed seed for reproducible results
)

print(f"\nTrain size: {X_train.shape[0]} samples")
print(f"Test size: {X_test.shape[0]} samples")

**2.X: One-hot encoding**
Machine learning models like linear regression work with numbers, not strings.  
We therefore need to convert categorical features into a numeric format that a ML model can understand.

We use:

- **`ColumnTransformer` with `OneHotEncoder`**  
  - Each categorical column is expanded into one binary column per category.
  - `handle_unknown='ignore'` ensures the model can handle categories that only appear in the test set.

In [ ]:
# One-hot encode the categorical columns and keep numeric ones as they are.
# The preprocessor will be reused for all downstream models.

preprocessor = ColumnTransformer(
    transformers=[
        # 'cat' transformer:
        #   - OneHotEncoder turns each category into a binary column
        #   - handle_unknown='ignore' lets us safely see unseen categories in test data
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_for_model_consolidated)
    ],
    remainder='passthrough'  # numerical columns go through unchanged
)

# 1. Fit preprocessor ONLY on training data to avoid data leakage
preprocessor.fit(X_train)

# 2. Transform train and test sets into numeric matrices
X_train_enc = preprocessor.transform(X_train)
X_test_enc  = preprocessor.transform(X_test)

## 3. Choose families of models & hyperparameter variations to test:

1. Linear Regression
    backward & forward feature reduction
2. Ridge regression
3. Neural Networks (MLP)  
    3.1  
    3.2  
    3.3  
    3.4  


## 4. Execute (train and evaluate) chosen models:

**4.1: Linear Regression**

We start with a plain **linear regression** model using all encoded features:
- Input: one-hot encoded categorical features + numerical features  
- Goal: find coefficients that best fit `combination_mpg` in a least-squares sense  
- Metric: **R² on the test set**, which tells us how much of the variance in MPG is explained.


In [ ]:
# 1. Create a plain linear regression estimator
model_LR = LinearRegression()

# 2. Fit the model on the encoded training data
model_LR.fit(X_train_enc, y_train)

# 3. Predict on the held-out test data
y_pred = model_LR.predict(X_test_enc)

# 4. Compute R² on the test set (how much variance in y we explain)
r2 = r2_score(y_test, y_pred)
print(f"\nLinear Regression R² Score on test set: {r2:.3f}")


In [ ]:
# Visualizing linear-regression performance

# 1. Predicted vs actual MPG
plt.figure(figsize=(6, 4))
plt.scatter(y_test, y_pred, alpha=0.5)
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], linestyle="--")
plt.xlabel("Actual MPG (test set)")
plt.ylabel("Predicted MPG")
plt.title("Linear Regression: Actual vs Predicted MPG")
plt.tight_layout()
plt.show()

# 2. Residuals plot (errors vs prediction)
residuals = y_test - y_pred

plt.figure(figsize=(6, 4))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, linestyle="--")
plt.xlabel("Predicted MPG")
plt.ylabel("Residual (Actual - Predicted)")
plt.title("Linear Regression: Residuals vs Predicted")
plt.tight_layout()
plt.show()


**4.1.1: Linear Regression with forward feature selection**

We apply forward feature selection on top of linear regression:

- Start with no features, then greedily add the feature that most improves cross-validated R².
- Stop when we reach `n_select` features.
- Retrain a linear regression model on this reduced feature set and compare performance.

In [73]:
n_select = X_train_enc.shape[1] - 8 # number of features to keep

estimator = model_LR
#forward selection
# we could use the cv cross validation parameter as a hyperparameter to optimize/play with
sfs_forward = SequentialFeatureSelector(estimator, n_features_to_select=n_select, direction='forward',cv=5)

sfs_forward.fit(X_train_enc,y_train)
# Extract mask of selected features
selected_mask = sfs_forward.get_support()

# Reduce train/test matrices
X_train_fs = X_train_enc[:, selected_mask]
X_test_fs  = X_test_enc[:, selected_mask]

# Fit a new model on selected features
lr_fs = LinearRegression()
lr_fs.fit(X_train_fs, y_train)

# Predict + evaluate
y_pred_forward = lr_fs.predict(X_test_fs)
r2_forward = r2_score(y_test, y_pred_forward)
print(f"\nForward Feature Selection Linear Regression R² Score on test set: {r2_forward:.3f}")

#Get feature names
feature_names = preprocessor.get_feature_names_out()
# print(feature_names)
# Extract selected and removed names
selected_features = feature_names[selected_mask]
removed_features  = feature_names[~selected_mask]

print("\nSelected features:")
for f in selected_features:
    print("   ", f)

print("\nRemoved features:")
for f in removed_features:
    print("   ", f)




Forward Feature Selection Linear Regression R² Score on test set: 0.665

Selected features:
    cat__size_medium
    cat__drive_4wd
    cat__drive_awd
    cat__drive_fwd
    cat__fuel_type_electricity
    cat__fuel_type_gas
    cat__transmission_a
    remainder__cylinders
    remainder__displacement

Removed features:
    cat__size_large
    cat__size_small
    cat__drive_rwd
    cat__fuel_type_diesel
    cat__make_region_america
    cat__make_region_asia
    cat__make_region_europe
    cat__transmission_m


In [69]:
print(X_train_enc.shape[1])#-np.linspace(3,14,num=12))

17


In [ ]:
param_grid = {'sfs__n_features_to_select': (np.linspace(3,14,num=12)).astype(int)}

model = Pipeline([('sfs',SequentialFeatureSelector(estimator, direction='forward', cv=5)),('lr',estimator)])

grid = GridSearchCV(model, param_grid, cv=5, scoring='r2')
grid.fit(X_train_enc, y_train)

n_select = grid.best_params_

print("Best alpha after parameter optimization: alpha = ",n_select)
print("Best cross-validation parameter sweep R²:", grid.best_score_)




In [ ]:
# running forward FS with optimal number of parameters to remove

estimator = model_LR
#forward selection
# we could use the cv cross validation parameter as a hyperparameter to optimize/play with
sfs_forward = SequentialFeatureSelector(estimator, n_features_to_select=n_select, direction='forward',cv=5)

sfs_forward.fit(X_train_enc,y_train)
# Extract mask of selected features
selected_mask = sfs_forward.get_support()

# Reduce train/test matrices
X_train_fs = X_train_enc[:, selected_mask]
X_test_fs  = X_test_enc[:, selected_mask]

# Fit a new model on selected features
lr_fs = LinearRegression()
lr_fs.fit(X_train_fs, y_train)

# Predict + evaluate
y_pred_forward = lr_fs.predict(X_test_fs)
r2_forward = r2_score(y_test, y_pred_forward)
print(f"\nForward Feature Selection Linear Regression R² Score on test set: {r2_forward:.3f}")

#Get feature names
feature_names = preprocessor.get_feature_names_out()
# print(feature_names)
# Extract selected and removed names
selected_features = feature_names[selected_mask]
removed_features  = feature_names[~selected_mask]

print("\nSelected features:")
for f in selected_features:
    print("   ", f)

print("\nRemoved features:")
for f in removed_features:
    print("   ", f)



**4.1.2 Backward Feature Selection LR**
We also try Ridge Regression, which is linear regression with L2 penalty:

- Adds a penalty on large coefficients to reduce overfitting.
- Controlled by hyperparameter α.
- We use cross-validation over several α values and choose the one that gives the best average R².

In [ ]:
n_select = X_train_enc.shape[1] - 8 # number of features to keep

estimator = model_LR
# Backward selection
# We could use the cv cross validation parameter as a hyperparameter to optimize/play with
sfs_backward = SequentialFeatureSelector(estimator, n_features_to_select=n_select, direction='backward',cv=5)

sfs_backward.fit(X_train_enc,y_train)
# Extract mask of selected features
selected_mask = sfs_backward.get_support()

# Reduce train/test matrices
X_train_fs = X_train_enc[:, selected_mask]
X_test_fs  = X_test_enc[:, selected_mask]

# Fit a new model on selected features
lr_fs = LinearRegression()
lr_fs.fit(X_train_fs, y_train)

# Predict + evaluate
y_pred_backward = lr_fs.predict(X_test_fs)
r2_backward = r2_score(y_test, y_pred_backward)
print(f"\nbackward Feature Selection Linear Regression R² Score on test set: {r2_backward:.3f}")

# Get feature names 
feature_names = preprocessor.get_feature_names_out()
# print(feature_names)
# Extract selected and removed names
selected_features = feature_names[selected_mask]
removed_features  = feature_names[~selected_mask]

print("\nSelected features:")
for f in selected_features:
    print("   ", f)

print("\nRemoved features:")
for f in removed_features:
    print("   ", f)



**4.2 Ridge Regression**

In [ ]:
# Hyperparameter search for Ridge Regression (L2-regularized linear model)
# We search over different alpha values using 5-fold cross-validation

param_grid = {'alpha': [0.001,0.01, 0.1, 1, 10, 100]}
param_grid = {'alpha': np.logspace(-2,-1,num=10)}

model_ridge = Ridge()

grid = GridSearchCV(model_ridge, param_grid, cv=5, scoring='r2')
grid.fit(X_train_enc, y_train)


best_alpha = grid.best_params_['alpha']

print("Best alpha after parameter optimization: alpha = ", best_alpha)
print("Best cross-validation parameter sweep R²:", grid.best_score_)

# creating model_ridge with the optimized alpha value
model_ridge_best = Ridge(alpha=best_alpha)
model_ridge_best.fit(X_train_enc, y_train)

y_pred = model_ridge_best.predict(X_test_enc)
r2 = r2_score(y_test, y_pred)
print("Best Ridge Test R²:", r2)


**4.X: Neural Networks**

In [ ]:
print("Final input feature dimension:", X_train_enc.shape[1])

**9. Neural Network Architectures**

To explore non‑linear relationships, we implement four different neural network models:

- **Model A: Simple**:  
  - 1 hidden layer with 32 neurons (ReLU)  
  - Good baseline, fast to train

- **Model B: Deeper**:  
  - Two hidden layers (64 → 32 units)  
  - CAN model, more complex patterns

- **Model C: Regularized with Dropout**:  
  - Same idea as Model B but adds `Dropout(0.25)`  
  - Helps reduce overfitting by randomly dropping neurons during training

- **Model D: Wide with Batch Normalization**:  
  - Two wide layers (128 → 64 units)  
  - `BatchNormalization` after each hidd**en layer  
  - Makes training more stable and can speed up convergence

All models use:
- **Optimizer**: Adam
- **Loss**: Mean Squared Error (MSE)
- **Metric**: Mean Absolute Error (MAE), easier to interpret as “average error in MPG”

In [ ]:
# NN models

def build_model_A(input_dim):
    """Model A - Simple 1-hidden-layer network."""
    model = Sequential([
        Dense(32, activation='relu', input_dim=input_dim),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


def build_model_B(input_dim):
    """Model B - Deeper network with two hidden layers."""
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


def build_model_C(input_dim):
    """Model C - Adds dropout for regularization."""
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.25),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


def build_model_D(input_dim):
    """Model D - Wide network using batch normalization."""
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

We train each neural network using:

- Early stopping (`EarlyStopping`) with:  
  - `patience=10`: if validation loss does not improve for 10 epochs, training stops  
  - `restore_best_weights=True`: we keep the model from the best epoch

For each model we:

1. Train on the encoded training data with a validation split.
2. Evaluate on the encoded test set to obtain MSE  and MAE.
3. Plot the training and validation loss curves to see whether the model is overfitting or underfitting.

This makes the comparison between different architectures transparent and well‑documented.

In [ ]:
# Train and evaluate each neural network variant

callbacks = [
    # Early stopping stops training if val loss doesn't improve for 10 epochs,
    # and restores the weights from the best epoch.
    EarlyStopping(patience=10, restore_best_weights=True)
]

# Build one instance of each model architecture
models = {
    "Model A": build_model_A(X_train_enc.shape[1]),
    "Model B": build_model_B(X_train_enc.shape[1]),
    "Model C": build_model_C(X_train_enc.shape[1]),
    "Model D": build_model_D(X_train_enc.shape[1]),
}

history = {}   # training curves (loss, val_loss)
results = {}   # final test metrics

for name, nn_model in models.items():
    print(f"Training {name}...")
    history[name] = nn_model.fit(
        X_train_enc, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=callbacks,
        verbose=0
    )
    # Evaluate on test data and store MSE/MAE
    test_mse, test_mae = nn_model.evaluate(X_test_enc, y_test, verbose=0)
    results[name] = (test_mse, test_mae)
    print(f"{name} — Test MAE: {test_mae:.3f}")


**Visualizing Training Curves**  
Finally, we plot the training loss and validation loss for each model across epochs.

Things to look for:

- If validation loss starts increasing while training loss keeps decreasing, then the model is overfitted.  
- If both losses are high and do not improve, then the model might be underpowered or features might not be informative enough

These plots help justify which neural network architectures are the most appropriate for this prediction task.

In [ ]:
# Plot training curves for each model
for name in history:
    plt.figure(figsize=(6, 4))
    plt.plot(history[name].history['loss'], label='Train Loss')
    plt.plot(history[name].history['val_loss'], label='Val Loss')
    plt.title(name)
    plt.xlabel("Epochs")
    plt.ylabel("MSE")
    plt.legend()
    plt.show()

## 5: Choose best model and evaluate with test data:

In this final section we compare all candidate models on the held-out test set and pick a
single “best” model for predicting combined MPG.

We focus on:

- **R²** – how much of the variance in MPG the model explains  
- **MAE** – average absolute error in MPG (easier to interpret)  
- **RMSE** – root mean squared error (penalizes large mistakes)

Models we compare:

1. **Baseline Linear Regression** (all encoded features)  
2. **Ridge Regression** with the best α value (from Section 4.5)  
3. **Neural Networks (Models A–D)** from Section 4.X  

By evaluating all models on the **same test set**, we can see which one generalizes best,
instead of just memorizing the training data.


In [ ]:
results_summary = []

# 1. Plain Linear Regression
y_pred_lr = model_LR.predict(X_test_enc)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)

results_summary.append({
    "Model": "Linear Regression",
    "Family": "Linear",
    "R2": r2_score(y_test, y_pred_lr),
    "MAE": mean_absolute_error(y_test, y_pred_lr),
    "RMSE": rmse_lr,
})

# 2. Best Ridge Regression
# model_ridge_best = Ridge(alpha=best_alpha)
model_ridge_best.fit(X_train_enc, y_train)
y_pred_ridge = model_ridge_best.predict(X_test_enc)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)

results_summary.append({
    "Model": f"Ridge (alpha={best_alpha})",
    "Family": "Linear (regularized)",
    "R2": r2_score(y_test, y_pred_ridge),
    "MAE": mean_absolute_error(y_test, y_pred_ridge),
    "RMSE": rmse_ridge,
})

# 3. Neural Networks A–D
for name, nn_model in models.items():
    y_pred_nn = nn_model.predict(X_test_enc).flatten()
    mse_nn = mean_squared_error(y_test, y_pred_nn)
    rmse_nn = np.sqrt(mse_nn)

    results_summary.append({
        "Model": name,
        "Family": "Neural Network",
        "R2": r2_score(y_test, y_pred_nn),
        "MAE": mean_absolute_error(y_test, y_pred_nn),
        "RMSE": rmse_nn,
    })

results_df = pd.DataFrame(results_summary).sort_values(by="MAE").reset_index(drop=True)

print("Test-set performance of all models:\n")
display(results_df)

best_row = results_df.iloc[0]
print("\nBest overall model on the test set:")
print(f"  {best_row['Model']} "
      f"(Family: {best_row['Family']}) "
      f"— R²={best_row['R2']:.3f}, MAE={best_row['MAE']:.3f}, RMSE={best_row['RMSE']:.3f}")


The scores show how well each model handled the task of predicting a vehicle’s combined MPG:
- R² tells us how much of the variation in fuel economy the model can explain — higher is better.
- MAE gives the average size of the model’s mistakes in MPG, which is easy to understand in real-world terms.
- RMSE punishes bigger errors more heavily and is a good indicator of how stable the model is across different kinds of cars.

Comparing these models side-by-side reveals which approach generalized best rather than simply memorizing the training data.
The best model delivers the most reliable MPG predictions and is the that would be best for deployment or continued focused refinement.